<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/DAPT_TAPT_seq_bn_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

dataset = load_dataset(dataset_name)

print(dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 44876
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9754
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1980
    })
})


In [8]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

adapter_config = SeqBnConfig()
adapter_name = "dapt_tapt_seq_bn"
adapter_hub_name = f"pretrain_{adapter_name}_adpater"

In [10]:
print(adapter_hub_name)

pretrain_dapt_tapt_seq_bn_adpater


In [11]:
dapt_adapter_name = "jgrc3/RobertaDAPT_adapters_pfeiffer_reDo"

In [12]:
# Add a new adapter
loaded_name = model.load_adapter(dapt_adapter_name, config=adapter_config)

# Set the adapter to be used for training
model.train_adapter(loaded_name)

print(f"Loaded Adapter Name: {loaded_name}")

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Loaded Adapter Name: review_adapter_pfeiffer


In [13]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
review_adapter_pfeiffer  bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [16]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="dapt_tapt_seq_bn_helpfulness_pretrained_model",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [17]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [20]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=pretraining_args,
    train_dataset=dataset["test"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

In [21]:
pretrain_results = trainer.train()
print(pretrain_results)

Epoch,Training Loss,Validation Loss
0,1.675400,1.563307
2,1.643500,1.548594
4,1.630600,1.532769
6,1.619200,1.518827
8,1.607600,1.505770
10,1.602000,1.501838
12,1.591500,1.497817
14,1.589000,1.506844
16,1.587200,1.488381
18,1.580500,1.502985


TrainOutput(global_step=14647, training_loss=1.5669949035998734, metrics={'train_runtime': 15348.4591, 'train_samples_per_second': 63.55, 'train_steps_per_second': 1.512, 'total_flos': 1.6458068922017587e+17, 'train_loss': 1.5669949035998734, 'epoch': 63.0})


In [22]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 1.4369633197784424, 'eval_runtime': 108.9652, 'eval_samples_per_second': 89.515, 'eval_steps_per_second': 4.267, 'epoch': 63.0}


In [1]:
model.push_adapter_to_hub(
    adapter_hub_name,
    loaded_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

NameError: name 'model' is not defined

In [ ]:
trainer.push_to_hub()

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()